   # On storing missing values in InfluxDB

Usually `NaN` or `null` are used to indicate missing values in tabular data. InfluxDB does not support `NaN` or `null` values and it has been subject of long debates in https://github.com/influxdata/influxdb/issues. 

In this notebook we investigate how InfluxDB handles missing values and then propose an approch to handle `NaN` or `null` values present in the data we want to store in InfluxDB.

The short answer is "missing data are not stored in InfluxDB".

We are going to use `requests` as the most fundamental method to interact with the InfluxDB HTTP API. During this investigation, it was found that the existing Python clients `influxdb-python` and `aioinflux` do not handle missing values correctly.

In [1]:
import requests

# Syntax
The InfluxDB [line protocol](https://docs.influxdata.com/influxdb/v1.7/write_protocols/line_protocol_reference/) has the following syntax:

`<measurement>[,<tag_key>=<tag_value>[,<tag_key>=<tag_value>]] <field_key>=<field_value>[,<field_key>=<field_value>] [<timestamp>]`

Conceptually you can think of a measurement as an SQL table, where the primary index is always time. tags and fields are effectively columns in the table. tags are indexed, and fields are not. The difference is that, with InfluxDB, you can have millions of measurements, you don’t have to define schemas up-front, and **null values aren’t stored**.

Also,  if you don't specify a timestamp InfluxDB uses the server’s local nanosecond timestamp in Unix epoch. 

To verify that we cannot store a `NaN` or `null` in InfluxDB let's start by sending a point cotaining those values. For simplicity, the data contains only a measurement and a field.

In [3]:
r = requests.post(url="https://influxdb-demo.lsst.codes/write?db=mydb", 
                  data='mytopic myfield=NaN')
if r.status_code != '204':
    print(r.text)

{"error":"unable to parse 'mytopic myfield=NaN': invalid number"}



In [4]:
r = requests.post(url="https://influxdb-demo.lsst.codes/write?db=mydb", 
                  data='mytopic myfield=null')
if r.status_code != '204':
    print(r.text)

{"error":"unable to parse 'mytopic myfield=null': invalid number"}



Now let's add a couple of valid numbers at different times:

In [5]:
r = requests.post(url="https://influxdb-demo.lsst.codes/write?db=mydb", 
                  data='mytopic myfield=1.0')
if r.status_code != '204':
    print(r.text)

In [6]:
r = requests.post(url="https://influxdb-demo.lsst.codes/write?db=mydb", 
                  data='mytopic myfield=2.0')
if r.status_code != '204':
    print(r.text)

You can retrieve the data with:

In [7]:
r = requests.get(url="https://influxdb-demo.lsst.codes/query", 
                 params={'q':'SELECT * FROM "mydb"."autogen"."mytopic"'})
r.json()

{'results': [{'statement_id': 0,
   'series': [{'name': 'mytopic',
     'columns': ['time', 'myfield'],
     'values': [['2019-09-10T03:59:30.976287925Z', 1],
      ['2019-09-10T03:59:33.542259527Z', 2]]}]}]}

Let's add a second field to the same measurement:

In [8]:
r = requests.post(url="https://influxdb-demo.lsst.codes/write?db=mydb", 
                  data='mytopic myfield=3.0,myotherfield=2.0')
r = requests.get(url="https://influxdb-demo.lsst.codes/query", 
                 params={'q':'SELECT * FROM "mydb"."autogen"."mytopic"'})
r.json()

{'results': [{'statement_id': 0,
   'series': [{'name': 'mytopic',
     'columns': ['time', 'myfield', 'myotherfield'],
     'values': [['2019-09-10T03:59:30.976287925Z', 1, None],
      ['2019-09-10T03:59:33.542259527Z', 2, None],
      ['2019-09-10T03:59:55.317192916Z', 3, 2]]}]}]}

After adding the new field, we imediatelly see that missing values for this field at previous times are returned as `None`. The same is true for future missing values of the intial field:

In [9]:
r = requests.post(url="https://influxdb-demo.lsst.codes/write?db=mydb", 
                  data='mytopic myotherfield=3.0')
r = requests.get(url="https://influxdb-demo.lsst.codes/query", 
                 params={'q':'SELECT * FROM "mydb"."autogen"."mytopic"'})
r.json()

{'results': [{'statement_id': 0,
   'series': [{'name': 'mytopic',
     'columns': ['time', 'myfield', 'myotherfield'],
     'values': [['2019-09-10T03:59:30.976287925Z', 1, None],
      ['2019-09-10T03:59:33.542259527Z', 2, None],
      ['2019-09-10T03:59:55.317192916Z', 3, 2],
      ['2019-09-10T04:00:38.816616908Z', None, 3]]}]}]}

In [11]:
r = requests.post(url="https://influxdb-demo.lsst.codes/write?db=mydb", 
                  data='mytopic myfield=5.0,myotherfield=4.0')
r = requests.get(url="https://influxdb-demo.lsst.codes/query", 
                 params={'q':'SELECT * FROM "mydb"."autogen"."mytopic"'})
r.json()

{'results': [{'statement_id': 0,
   'series': [{'name': 'mytopic',
     'columns': ['time', 'myfield', 'myotherfield'],
     'values': [['2019-09-10T03:59:30.976287925Z', 1, None],
      ['2019-09-10T03:59:33.542259527Z', 2, None],
      ['2019-09-10T03:59:55.317192916Z', 3, 2],
      ['2019-09-10T04:00:38.816616908Z', None, 3],
      ['2019-09-10T04:01:16.616654679Z', 5, 4]]}]}]}

# More on missing data

To understand why InfluxDB is different from other databases in this regard let's see its behaviour when data is sampled at different time intervals (a common feature of any time-series database).  The following InfluxQL query returns the mean value of `myfield` sampled every `15s`.  

In [12]:
r = requests.get(url="https://influxdb-demo.lsst.codes/query", 
                 params={'q':'SELECT mean(myfield) FROM "mydb"."autogen"."mytopic" GROUP BY time(15s)'})
r.json()

{'results': [{'statement_id': 0,
   'series': [{'name': 'mytopic',
     'columns': ['time', 'mean'],
     'values': [['2019-09-10T03:59:30Z', 1.5],
      ['2019-09-10T03:59:45Z', 3],
      ['2019-09-10T04:00:00Z', None],
      ['2019-09-10T04:00:15Z', None],
      ['2019-09-10T04:00:30Z', None],
      ['2019-09-10T04:00:45Z', None],
      ['2019-09-10T04:01:00Z', None],
      ['2019-09-10T04:01:15Z', 5]]}]}]}

Again, the default behaviour is to report `None` for time intervals with no data. 

InfuxQL (the InfluxDB query language) also provides a function to "fill" time intervals with no data, you can use `fill(none)` if you don't want to return missing values. 

In [13]:
r = requests.get(url="https://influxdb-demo.lsst.codes/query", 
                 params={'q':'SELECT mean(myfield) FROM "mydb"."autogen"."mytopic" GROUP BY time(15s) fill(none)'})
r.json()

{'results': [{'statement_id': 0,
   'series': [{'name': 'mytopic',
     'columns': ['time', 'mean'],
     'values': [['2019-09-10T03:59:30Z', 1.5],
      ['2019-09-10T03:59:45Z', 3],
      ['2019-09-10T04:01:15Z', 5]]}]}]}

You can also return the result of a linear interpolation for time intervals with no data using `fill(linear)`.

In [14]:
r = requests.get(url="https://influxdb-demo.lsst.codes/query", 
                 params={'q':'SELECT mean(myotherfield) FROM "mydb"."autogen"."mytopic" GROUP BY time(15s) fill(linear)'})
r.json()

{'results': [{'statement_id': 0,
   'series': [{'name': 'mytopic',
     'columns': ['time', 'mean'],
     'values': [['2019-09-10T03:59:45Z', 2],
      ['2019-09-10T04:00:00Z', 2.3333333333333335],
      ['2019-09-10T04:00:15Z', 2.6666666666666665],
      ['2019-09-10T04:00:30Z', 3],
      ['2019-09-10T04:00:45Z', 3.3333333333333335],
      ['2019-09-10T04:01:00Z', 3.6666666666666665],
      ['2019-09-10T04:01:15Z', 4],
      ['2019-09-10T04:01:30Z', None]]}]}]}

# How to handle `NaN` and `null` then?

The above should convince you that **we should not store any "sentinel values" for missing data** in InfluxDB replacing `NaN` or `null` which are not supported by constrution. The right aproach is to just drop the corresponding field when the incoming value is missing. In the case of the `lsst.sal.ATDome.logevent_azimuthCommandedState` topic that motivated this discussion:

`ATDomeID=1i,azimuth=\ufffd,commandedState=1i,priority=0i,private_host=1994757124i,private_kafkaStamp=1567888128.2887785,private_origin=32i,private_rcvStamp=1567888127.9708674,private_revCode="5544b90a",private_seqNum=1i,private_sndStamp=1567887306.2843742 1567888091465571857'`

the only place where it is possible to drop the `azimuth` field while keeping the others is in the InfluxDB Sink connector, right before writing to InfluxDB. The fact that the value is serialized to `\ufffd` instead seems to be a bug in the connector.  



